# Выполнили студенты гр. 6405-010302D:
- Сорокин Д.М.
- Буторина П.В.
- Морозов А.Е.

In [16]:
import numpy as np
from itertools import combinations

# 1.1 Реализовать функцию REF(), приводящую матрицу к ступенчатому виду.


In [17]:
def REF(matrix):
    mat = np.array(matrix)
    n_rows, n_cols = mat.shape
    lead = 0
    for r in range(n_rows):
        if lead >= n_cols:
            return mat
        i = r
        while mat[i, lead] == 0:
            i += 1
            if i == n_rows:
                i = r
                lead += 1
                if lead == n_cols:
                    return mat
        mat[[i, r]] = mat[[r, i]]

        for i in range(r + 1, n_rows):
            if mat[i, lead] != 0:
                mat[i] = (mat[i] + mat[r]) % 2
        lead += 1
    return mat

# 1.2. Реализовать функцию RREF(), приводящую матрицу к приведённому ступенчатому виду. 

In [18]:
def RREF(matrix):
    matrix = REF(matrix)
    n_rows, n_cols = matrix.shape

    for r in range(n_rows - 1, -1, -1):
        lead = np.argmax(matrix[r] != 0)
        if matrix[r, lead] != 0:
            for i in range(r - 1, -1, -1):
                if matrix[i, lead] != 0:
                    matrix[i] = (matrix[i] + matrix[r]) % 2
    while not any(matrix[n_rows - 1]):
        matrix = matrix[:-1, :]
        n_rows -= 1
    return matrix

# 1.3. Создать класс линейных кодов LinearCode

In [19]:
# класс линейных кодов
class LinearCode:
    def __init__(self, matrix):
        self.matrix = matrix
        self.G = self.delete_null_rows(RREF(matrix))
        self.lead_columns = self.get_lead_columns(self.G)
        self.X = self.delete_lead_columns(self.G, self.lead_columns)
        self.H = self.create_H_matrix(self.X, self.lead_columns)

    '''
       Функция очищает нулевые строки
    '''
    def delete_null_rows(self, matrix):
        # Определяем индексы строк, которые нужно удалить
        rows_to_delete = [i for i, row in enumerate(matrix) if np.sum(row) == 0]

        matrix = np.delete(matrix, rows_to_delete, axis=0)
        return matrix

    '''
       Функция получения ведущих столбцов
    '''
    def get_lead_columns(self, matrix):
        lead_columns = []
        for line in matrix:
            lead_columns.append(np.where(line == 1)[0][0])
        return lead_columns

    '''
        Функция удаления ведущих столбцов
    '''
    def delete_lead_columns(self, matrix, lead_columns):

        matrix = matrix.tolist()

        new_matrix = []

        for row in matrix:
            new_row = [element for j, element in enumerate(row) if j not in lead_columns]
            new_matrix.append(new_row)

        return np.array(new_matrix)

    '''
        Формирование проверочной матрицы
    '''
    def create_H_matrix(self, matrix, lead_columns):
        unit_matrix = np.eye(matrix.shape[1], dtype=int)
        result_matrix = np.zeros((matrix.shape[0] + unit_matrix.shape[0], matrix.shape[1]), dtype=int)
        idx_1 = 0
        idx_2 = 0

        for i in range(result_matrix.shape[0]):
            if i in lead_columns:
                result_matrix[i] = matrix[idx_1]
                idx_1 += 1
            else:
                result_matrix[i] = unit_matrix[idx_2]
                idx_2 += 1

        return result_matrix

    '''
        Формирование кодовых слов длиной n
        путем сложения слов
        из порождающего множества
    '''
    def generate_codewords_1(self):
        rows = self.G.shape[0]
        codewords = set()

        for r in range(1, rows + 1):
            for comb in combinations(range(rows), r):
                codeword = np.bitwise_xor.reduce(self.G[list(comb)], axis=0)
                codewords.add(tuple(codeword))

        codewords.add(tuple(np.zeros(self.G.shape[1], dtype=int)))

        return np.array(list(codewords))

    '''
        Формирование кодовых слов длиной n
        путем умножения всех
        двоичных слов длины k на G 
    '''
    def generate_codewords_2(self):
        k = self.G.shape[0]
        n = self.G.shape[1]
        codewords = []

        # Генерируем все двоичные слова длины k
        for i in range(2 ** k):
            binary_word = np.array(list(np.binary_repr(i, k)), dtype=int)
            codeword = np.dot(binary_word, self.G) % 2
            codewords.append(codeword)

        return np.array(codewords)

    '''
        Функция проверки кодовых слов
        путем умножения на проверочную матрицу
    '''
    def check_codeword(self, codeword):
        return np.dot(codeword, self.H) % 2

    '''
        Функция определения кодового расстояния
    '''
    def get_code_distance(self, codewords):
        if len(codewords) < 2:
            return 0

        min_distance = float('inf')

        for word1, word2 in combinations(codewords, 2):
            distance = np.sum(np.bitwise_xor(word1, word2))
            if distance > 0:
                min_distance = min(min_distance, distance)

        return min_distance

# Входные данные 

In [20]:
S = np.array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
              [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
              [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
              [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])

# Проверка работоспособности программы
### 1. Построение необходимых матриц

In [21]:
print("Исходная матрица:\n", S)

# Инициализация класса линейных кодов
linearCode = LinearCode(S)

print("\nG* матрица:\n", linearCode.G)
print("\nВедущие столбцы: ", linearCode.lead_columns)
print("\nСокращенная матрица X:\n", linearCode.X)
print("\nПроверочная матрица H:\n", linearCode.H)

Исходная матрица:
 [[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 1 0 0 1 1 1 0]
 [1 0 1 1 1 0 0 0 0 0 0]]

G* матрица:
 [[1 0 1 0 0 1 0 1 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]

Ведущие столбцы:  [0, 3, 4, 6, 8]

Сокращенная матрица X:
 [[0 1 1 1 1 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 1 1]]

Проверочная матрица H:
 [[0 1 1 1 1 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


### 2. Формирование кодовых слов и проверка на правильность

In [22]:
codewords_1 = linearCode.generate_codewords_1()
codewords_2 = linearCode.generate_codewords_2()
print("\nСформированные кодовые слова (1 способ):\n", codewords_1)
print("\nСформированные кодовые слова (2 способ):\n", codewords_2)

# Проверяем списки кодовых слов на совпадение
assert set(map(tuple, codewords_1)) == set(
    map(tuple, codewords_2)), "Наборы кодовых слов не совпадают!"

# Проверка кодовых слов с помощью матрицы H
for codeword in codewords_2:
    result = linearCode.check_codeword(codeword)
    assert np.all(result == 0), f"Ошибка: кодовое слово {codeword} не прошло проверку матрицей H"

print("\nКодовые слова прошли проверку.\n")


Сформированные кодовые слова (1 способ):
 [[0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 1 1 1 1 1 1 1 1]
 [1 0 1 0 1 1 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [1 0 1 1 1 0 1 0 0 1 0]
 [1 0 1 1 0 0 1 1 1 0 0]
 [0 0 0 0 1 0 0 1 1 1 0]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 1 1 1 1 1 0 0 0]
 [0 0 0 0 1 0 1 1 1 0 0]
 [1 0 1 1 0 0 1 1 0 1 1]
 [0 0 0 1 1 1 0 1 1 0 1]
 [1 0 1 0 0 1 1 1 0 0 0]
 [1 0 1 0 0 1 1 1 1 1 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 0 0 0 1 1]
 [1 0 1 0 0 1 0 1 0 1 0]
 [1 0 1 1 1 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 1 1 0]
 [1 0 1 1 0 0 0 1 1 1 0]
 [1 0 1 0 1 1 1 0 0 0 1]
 [0 0 0 1 0 1 1 0 1 1 0]
 [1 0 1 0 0 1 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 0 0 1 1]]

Сформированные кодовые слова (2 способ):
 [[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 1 0

### 3. Вычисление кодового расстояния и кратности обнаруживаемой ошибки

In [23]:
# Вычисляем кодовое расстояние и кратность обнаруживаемой ошибки
d = linearCode.get_code_distance(codewords_1)
print("d = ", d)

# t = (d - 1) // 2
t = (d - 1)
print("t = ", t)

d =  2
t =  1


### 4 .Проверка работы программы путем внесения ошибок

In [24]:
# Вносим в кодовое слово ошибку кратности не более t
# Необходимо убедиться в обнаружении ошибки.
v = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0])
e1 = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
v_e1 = (v + e1) % 2
print(f"e1 = {e1}")
print('v + e1 = ', v_e1)
print('(v + e1)@H =', linearCode.check_codeword(v_e1), "- error")

print()
# Найти для некоторого кодового слова ошибку кратности t+1
# такую, что при умножении на H ошибка не может быть обнаружена
e2 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])
print(f"e2 = {e2}")
v_e2 = (v + e2) % 2
print(f"v + e2 = {v_e2}")
print(f"(v + e2)@H = {linearCode.check_codeword(v_e2)} - no error")

e1 = [0 0 1 0 0 0 0 0 0 0 0]
v + e1 =  [1 0 0 1 1 0 1 0 0 1 0]
(v + e1)@H = [0 1 0 0 0 0] - error

e2 = [0 0 0 0 0 0 1 0 0 1 0]
v + e2 = [1 0 1 1 1 0 0 0 0 0 0]
(v + e2)@H = [0 0 0 0 0 0] - no error
